In [9]:
import gensim
import pandas as pd
import json
import math
import requests 

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)  

In [ ]:
model

In [1]:
import json

with open('./src/datasets.json','r') as f:
    datasets = json.load(f)
    
# datasets=datasets['results']

FileNotFoundError: [Errno 2] No such file or directory: './src/datasets.json'

In [12]:
page1 = requests.get('http://api.us.socrata.com/api/catalog/v1?domains=data.NJ.gov&search_context=data.NJ.GOV').json()
total = page1['resultSetSize']
pages = math.ceil(total/100)
results = []
for page in range(pages):
    pageResults = requests.get(f'http://api.us.socrata.com/api/catalog/v1?domains=data.NJ.gov&search_context=data.NJ.GOV&offset={page*100}&limit=100').json()['results']
    results.extend(pageResults)
datasets= results

In [13]:
len(datasets)

492

In [14]:
lookup ={
}
rlookup={
}
def read_corpus(datasets, tokens_only=False):
    for index,dataset in enumerate(datasets):
            tokens = gensim.utils.simple_preprocess(dataset['resource']['name'] + " " + dataset['resource']['description'])
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                lookup[dataset['resource']['id']] = index
                rlookup[index] = dataset['resource']['id']
                yield gensim.models.doc2vec.TaggedDocument(tokens, [index])

train_corpus = list(read_corpus(datasets))
# test_corpus = list(read_corpus(lee_test_file, tokens_only=True))



In [15]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [16]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [17]:
vector = model.infer_vector(train_corpus[0].words)
vector

array([-0.13680468,  0.3919937 ,  0.00068124,  0.19463938,  0.16055907,
       -0.4844915 , -0.03056372,  0.1686794 ,  0.15181491,  0.06501225,
        0.09733491,  0.12010238,  0.16548626, -0.00288182,  0.00744367,
        0.14353925,  0.2714717 ,  0.47075793, -0.13730116, -0.06070143,
        0.14261241,  0.20349064,  0.29540244, -0.2549563 ,  0.02786489,
       -0.380275  ,  0.02617319, -0.11347429,  0.08431723,  0.04695805,
        0.01856744,  0.22756544,  0.07928962,  0.21188079, -0.21645992,
        0.4807366 ,  0.04655745,  0.24805148, -0.27978277, -0.04452246,
       -0.19175273, -0.53164047,  0.1932537 ,  0.244147  ,  0.11147839,
        0.05284307,  0.30567372,  0.28306463, -0.23946309, -0.09730349],
      dtype=float32)

In [18]:
datasets[0]['resource']['id']

'a7mk-8suc'

In [19]:
docForId('ic3t-wcy2')

NameError: name 'docForId' is not defined

In [20]:
a = [23,4,5]
b = [234,5]
a.extend(b)
a


[23, 4, 5, 234, 5]

In [21]:
results = {}
for datasetid in lookup.keys():
    inferred_vector = model.infer_vector(train_corpus[lookup[datasetid]].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=30)
    sims = [ {'dataset': rlookup[sim[0]], 'similarity': sim[1]} for sim in sims]
    results[datasetid] = sims

In [22]:
len(results.keys())

492

In [24]:
with open('public/NJ_similarity_metrics.json', 'w') as f:
    json.dump(results,f)

In [124]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [126]:
results['jign-uhe6']

[{'dataset': 'jign-uhe6', 'similarity': 0.6124996542930603},
 {'dataset': 'shpd-5q9m', 'similarity': 0.5990572571754456},
 {'dataset': 'i6bk-bwyv', 'similarity': 0.5953792333602905},
 {'dataset': 'i4kb-6ab6', 'similarity': 0.590259313583374},
 {'dataset': 'rgy2-tti8', 'similarity': 0.587500274181366},
 {'dataset': '2mhq-um7h', 'similarity': 0.5857139825820923},
 {'dataset': 'fx4z-5xg2', 'similarity': 0.5824402570724487},
 {'dataset': '799n-b76v', 'similarity': 0.5821795463562012},
 {'dataset': '9z9b-6hvk', 'similarity': 0.5805541276931763},
 {'dataset': 'c2g8-ercv', 'similarity': 0.579156756401062}]

In [125]:
results['mnz3-dyi8']

[{'dataset': 'jign-uhe6', 'similarity': 0.6124996542930603},
 {'dataset': 'shpd-5q9m', 'similarity': 0.5990572571754456},
 {'dataset': 'i6bk-bwyv', 'similarity': 0.5953792333602905},
 {'dataset': 'i4kb-6ab6', 'similarity': 0.590259313583374},
 {'dataset': 'rgy2-tti8', 'similarity': 0.587500274181366},
 {'dataset': '2mhq-um7h', 'similarity': 0.5857139825820923},
 {'dataset': 'fx4z-5xg2', 'similarity': 0.5824402570724487},
 {'dataset': '799n-b76v', 'similarity': 0.5821795463562012},
 {'dataset': '9z9b-6hvk', 'similarity': 0.5805541276931763},
 {'dataset': 'c2g8-ercv', 'similarity': 0.579156756401062}]

In [105]:
len(datasets)

100

## All Domains

In [138]:
from urllib.request import urlretrieve

In [139]:
import requests

In [147]:
domains = requests.get("http://api.us.socrata.com/api/catalog/v1/domains").json()['results']

In [160]:

def get_dataset_list_chunk(domain, page,chunk_size):
    offset= page*chunk_size
    limit = chunk_size
    url='https://api.us.socrata.com/api/catalog/v1?domains={domain}&search_context={domain}&offset={offset}&limit={limit}'
    return requests.get(url.format(domain=domain, offset=offset,limit=limit)).json()['results']

In [ ]:
chunk_size=100

results={}
for domain in domains:
    datasets = []
    print('doing ',domain['domain'])
    total = domain['count']
    for i in range(total//chunk_size +1): 
        datasets.extend(get_dataset_list_chunk(domain['domain'], 0 ,100))
    results[domain['domain']]=datasets

In [166]:
with open('all_portal_datasets.json', 'w') as f:
    json.dump(results,f)

In [3]:
with open('all_portal_datasets.json') as f:
    results = json.load(f)

## All dataset document modeling

In [4]:
all_data_lookup={}
reverse_all_data_lookup={}
index = 0

def read_corpus():
    for domain in results.keys():
        for dataset in results[domain]:
            dataset_id = dataset['resource']['id']
            key = "{domain}_{did}".format(domain=domain, did=dataset_id)
            all_data_lookup[key] = index 
            reverse_all_data_lookup[index] = key
            description = dataset['resource']['description']
            tokens = gensim.utils.simple_preprocess(dataset['resource']['name'] + " " + dataset['resource']['description'])
            yield gensim.models.doc2vec.TaggedDocument(tokens, [index])
train_corpus = list(read_corpus())


In [ ]:
all_model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
all_model.build_vocab(train_corpus)
all_model.train(train_corpus, total_examples=all_model.corpus_count, epochs=all_model.epochs)

## 